In [1]:
from sklearn.model_selection import KFold, cross_val_predict
from itertools import combinations
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import time
import pickle
import warnings
warnings.filterwarnings('ignore')



In [2]:
#load data
with open('../Features/Fingerprints/fingerprints_df_grouped.pickle', 'rb') as f:
    fingerprints = pickle.load(f)

with open('../Data/processed/clean_df_grouped.pkl', 'rb') as f:
    df = pickle.load(f)

    
df.reset_index(drop=True, inplace=True)

In [3]:
with open('fingerprint_results.pickle', 'rb') as f:
    fingerprint_results = pickle.load(f)


In [4]:
fingerprints = dict(sorted(fingerprints.items()))
fingerprints.keys()

dict_keys(['ankh_base_embedding', 'atompair', 'atompair-count', 'avalon', 'avalon-count', 'binary profile of physicochemical property', 'ecfp', 'ecfp-count', 'erg', 'estate', 'fcfp', 'fcfp-count', 'layered', 'maccs', 'map4', 'mean_embeddings_large', 'one-hot-encoded-sequence', 'pattern', 'peptide_descriptors', 'rdkit', 'rdkit-count', 'secfp', 'topological', 'topological-count'])

In [5]:
fingerprints_filtered = {
    'erg': fingerprints['erg'],
    'atompair-count': fingerprints['atompair-count'],
    'ankh_base_embedding': fingerprints['ankh_base_embedding'],
}
fingerprints_filtered = fingerprints 
variants = ['GpTx-1', 'Protoxin II', 'JzTx-V', 'Huwentoxin-IV']


In [10]:
enc = OneHotEncoder()
one_hot_features = enc.fit_transform(df[['Assay', 'REGION_NOTES']]).toarray()
fingerprint_results = {}  # Dictionary to store the results of each fingerprint
variant_r2_scores = {}
predictions_dict = {}


def train_and_predict_model(X, y, model_type):
    start_time_model = time.time()
    if model_type == 'ankh_base_embedding':
        print("Training Ridge model for ankh_base_embedding")
        model = Ridge()
    else:
        print(f"Training HistGradientBoostingRegressor model for {model_type}")
        model = HistGradientBoostingRegressor()
    predictions = cross_val_predict(model, X, y, cv=kf, n_jobs=-1)
    end_time_model = time.time()
    time_taken_model = end_time_model - start_time_model
    print(f"Finished training model for {model_type}. Time taken: {time_taken_model} seconds")
    return predictions

kf = KFold(n_splits=5, shuffle=True, random_state=123)
for variant in variants:
    variant_r2_scores[variant] = {}

start_time_subset = time.time()

for fingerprint_name, fingerprint in fingerprints_filtered.items():
    X = pd.DataFrame(fingerprint).loc[df.index]
    X = pd.concat([X, pd.DataFrame(one_hot_features).loc[df.index]], axis=1)
    y = df['pIC50']
    predictions = train_and_predict_model(X, y, fingerprint_name)
    predictions_dict[fingerprint_name] = predictions
    fingerprint_results[fingerprint_name] = r2_score(y,predictions)  # Store the results in the dictionary
    for variant in variants:
        variant_df = df[df['Variants_of'] == variant]
        y_true = variant_df['pIC50']
        y_pred = pd.DataFrame(predictions, index=df.index).loc[variant_df.index]
        variant_r2_scores[variant][fingerprint_name] = r2_score(y_true, y_pred)

end_time_subset = time.time()
time_taken_subset = end_time_subset - start_time_subset
print(f"Finished processing. Time taken: {time_taken_subset} seconds")


Training HistGradientBoostingRegressor model for erg
Finished training model for erg. Time taken: 3.3010239601135254 seconds
Training HistGradientBoostingRegressor model for atompair-count
Finished training model for atompair-count. Time taken: 21.775076150894165 seconds
Training Ridge model for ankh_base_embedding
Finished training model for ankh_base_embedding. Time taken: 2.627450704574585 seconds
Finished processing. Time taken: 40.15343999862671 seconds


In [25]:
fingerprint_results = dict(sorted(fingerprint_results.items(), key=lambda item: item[1], reverse=True))
fingerprint_results

{'atompair-count': 0.7555486004829163,
 'erg': 0.7479860640974708,
 'ankh_base_embedding': 0.7475686134581483,
 'mean_embeddings_large': 0.744434056583875,
 'avalon-count': 0.7289049132351113,
 'fcfp-count': 0.7216412323635801,
 'atompair': 0.7215049378404939,
 'rdkit-count': 0.7201422333705675,
 'ecfp-count': 0.7197040482368854,
 'binary profile of physicochemical property': 0.7022586910356519,
 'topological-count': 0.6971862322365004,
 'estate': 0.6931181352659332,
 'topological': 0.6869440909280372,
 'one-hot-encoded-sequence': 0.6857660778191714,
 'peptide_descriptors': 0.6839214004092842,
 'avalon': 0.6494795378431328,
 'secfp': 0.6405243047967196,
 'ecfp': 0.6393152666077775,
 'pattern': 0.6214749706101813,
 'fcfp': 0.6174737802847448,
 'rdkit': 0.5893495771125236,
 'layered': 0.5883275349313508,
 'maccs': 0.518314065542206,
 'map4': 0.39648485554937385}

In [54]:
for fingerprint_name, r2_value in fingerprint_results.items():
    print(f"{fingerprint_name}: {round(r2_value,2)}")

atompair-count: 0.76
erg: 0.75
ankh_base_embedding: 0.75
mean_embeddings_large: 0.74
avalon-count: 0.73
fcfp-count: 0.72
atompair: 0.72
rdkit-count: 0.72
ecfp-count: 0.72
binary profile of physicochemical property: 0.7
topological-count: 0.7
estate: 0.69
topological: 0.69
one-hot-encoded-sequence: 0.69
peptide_descriptors: 0.68
avalon: 0.65
secfp: 0.64
ecfp: 0.64
pattern: 0.62
fcfp: 0.62
rdkit: 0.59
layered: 0.59
maccs: 0.52
map4: 0.4


In [ ]:
#sort the results
#fingerprint_results = dict(sorted(fingerprint_results.items(), key=lambda item: item[1], reverse=True))
fingerprint_results

{'atompair-count': 0.7713282708528858,
 'mean_embeddings_large': 0.7561790175819262,
 'ankh_base_embedding': 0.7548104203141552,
 'erg': 0.7529567083023275,
 'avalon-count': 0.7434874941775433,
 'rdkit-count': 0.7357156161182916,
 'atompair': 0.7347121355640724,
 'ecfp-count': 0.7302001775949873,
 'fcfp-count': 0.7280318865404167,
 'topological-count': 0.7206112684363487,
 'binary profile of physicochemical property': 0.717640337071934,
 'estate': 0.7049121007794572,
 'topological': 0.7001100584558153,
 'peptide_descriptors': 0.6921989124327153,
 'one-hot-encoded-sequence': 0.6914689127156851,
 'avalon': 0.654947693755729,
 'ecfp': 0.6433126134690843,
 'secfp': 0.6419157909096964,
 'pattern': 0.6370518706731327,
 'fcfp': 0.6221165051971451,
 'rdkit': 0.5995481882527592,
 'layered': 0.5946549442042812,
 'maccs': 0.5136941953214202,
 'map4': 0.40258354903657}

In [11]:
#ensemble
from sklearn.metrics import r2_score

for i in range(1, 11):
    embedding_predictions = []
    y = df['pIC50']
    for fingerprint_name in list(fingerprint_results.keys())[:i]:
        embedding_predictions.append(predictions_dict[fingerprint_name])
    mean_predictions = np.mean(embedding_predictions, axis=0)
    print(f"Ensemble of {i} fingerprints")
    print(f"Fingeprints used: {list(fingerprint_results.keys())[:i]}")
    print(f"R2 score: {round(r2_score(y, mean_predictions),2)}")
    print('----------------------------------')

Ensemble of 1 fingerprints
Fingeprints used: ['erg']
R2 score: 0.75
----------------------------------
Ensemble of 2 fingerprints
Fingeprints used: ['erg', 'atompair-count']
R2 score: 0.77
----------------------------------
Ensemble of 3 fingerprints
Fingeprints used: ['erg', 'atompair-count', 'ankh_base_embedding']
R2 score: 0.79
----------------------------------
Ensemble of 4 fingerprints
Fingeprints used: ['erg', 'atompair-count', 'ankh_base_embedding']
R2 score: 0.79
----------------------------------
Ensemble of 5 fingerprints
Fingeprints used: ['erg', 'atompair-count', 'ankh_base_embedding']
R2 score: 0.79
----------------------------------
Ensemble of 6 fingerprints
Fingeprints used: ['erg', 'atompair-count', 'ankh_base_embedding']
R2 score: 0.79
----------------------------------
Ensemble of 7 fingerprints
Fingeprints used: ['erg', 'atompair-count', 'ankh_base_embedding']
R2 score: 0.79
----------------------------------
Ensemble of 8 fingerprints
Fingeprints used: ['erg', 'at

In [ ]:
#write the prediction from ['atompair-count', 'erg', 'ankh_base_embedding'] to csv , include the variant name

predictions_df = df[['Variants_of', 'pIC50']]
predictions_df['Ensemble'] = (predictions_dict['atompair-count'] + predictions_dict['erg'] + predictions_dict['ankh_base_embedding']) / 3
predictions_df.to_csv('ensemble_predictions.csv', index=False)

In [53]:
variant_r2_scores
#for each variant , print the top 5 fingerprints
for variant in variants:
    print(f"Variant: {variant}")
    #first sort the fingerprints by r2 score
    variant_r2_scores[variant] = dict(sorted(variant_r2_scores[variant].items(), key=lambda item: item[1], reverse=True))
    print(f"Top 10 fingerprints: {list(variant_r2_scores[variant].keys())[:10]}")
    # print combined fingerprint name and r2 score
    print('----------------------------------')
    i = 0
    for fingerprint_name, r2_score in variant_r2_scores[variant].items():
            print(f"{fingerprint_name}: {round(r2_score, 2)}")
            i += 1
            if i == 10:
                i = 0
                break

    print('----------------------------------')

Variant: GpTx-1
Top 10 fingerprints: ['ankh_base_embedding', 'atompair-count', 'mean_embeddings_large', 'erg', 'fcfp-count', 'avalon-count', 'peptide_descriptors', 'ecfp-count', 'rdkit-count', 'estate']
----------------------------------
ankh_base_embedding: 0.8
atompair-count: 0.77
mean_embeddings_large: 0.77
erg: 0.76
fcfp-count: 0.75
avalon-count: 0.75
peptide_descriptors: 0.75
ecfp-count: 0.74
rdkit-count: 0.74
estate: 0.73
----------------------------------
Variant: Protoxin II
Top 10 fingerprints: ['erg', 'binary profile of physicochemical property', 'avalon-count', 'atompair-count', 'one-hot-encoded-sequence', 'ankh_base_embedding', 'mean_embeddings_large', 'rdkit-count', 'atompair', 'ecfp-count']
----------------------------------
erg: 0.69
binary profile of physicochemical property: 0.68
avalon-count: 0.67
atompair-count: 0.67
one-hot-encoded-sequence: 0.67
ankh_base_embedding: 0.66
mean_embeddings_large: 0.64
rdkit-count: 0.64
atompair: 0.62
ecfp-count: 0.59
-----------------

In [ ]:
variant_r2_scores

{'GpTx-1': {'map4': 0.49806559984876486,
  'erg': 0.7887046329531151,
  'atompair-count': 0.7915542485731139,
  'ecfp': 0.6751582784732595,
  'layered': 0.6058765378935222,
  'topological': 0.7247965309175233,
  'rdkit': 0.6215659941777497,
  'binary profile of physicochemical property': 0.7337567308481192,
  'one-hot-encoded-sequence': 0.6821196528097901,
  'peptide_descriptors': 0.7567251457637006,
  'ankh_base_embedding': 0.8087226740039888,
  'mean_embeddings_large': 0.7948957787103764,
  'maccs': 0.5509728888870917,
  'avalon': 0.6864672079554562,
  'fcfp': 0.659346396776398,
  'atompair': 0.7533630995108737,
  'pattern': 0.671727749228402,
  'secfp': 0.663560285187519,
  'estate': 0.7452512202932556,
  'avalon-count': 0.7728941252285998,
  'rdkit-count': 0.7632483600226732,
  'ecfp-count': 0.7627450062646608,
  'fcfp-count': 0.7654365433339878,
  'topological-count': 0.7508705506788922},
 'Protoxin II': {'map4': -0.06120071940928162,
  'erg': 0.6939563982139827,
  'atompair-count

In [ ]:
for variant in variant_r2_scores.keys():
    results = variant_r2_scores[variant]
    print(results)
    print("______________________")

0.7448399421053377
______________________
0.542264851070513
______________________
0.5592204418332176
______________________
0.20859755434589955
______________________


In [ ]:
with open('fingerprint_results.pickle', 'wb') as f:
    pickle.dump(fingerprint_results, f)